In [ ]:
!git clone https://github.com/kallioaa/frustration_msc.git
%cd /content/frustration_msc
import sys
sys.path.insert(0, "/content/frustration_msc/src")


In [ ]:
from pathlib import Path
import pickle

from experiments.sweep_plots import plot_sweep_training, plot_sweep_evaluation
from experiments.sweep_plots_helpper import (
    taxi_training_plot_specs,
    taxi_evaluation_plot_specs,
)

# Results

## Taxi V3

In [ ]:
# Taxi - confirmation bias
file_taxi_conf = Path("outputs/sweeps/taxi_final_results/taxi_conf_all_pairs.pkl")
with file_taxi_conf.open("rb") as f:
    conf_results = pickle.load(f)
print(f"Loaded {len(conf_results)} runs from {file_taxi_conf}")

Loaded 150 runs from outputs/sweeps/taxi_final_results/taxi_conf_all_pairs.pkl


In [ ]:
print(len(conf_results))

TypeError: object of type 'PosixPath' has no len()

In [ ]:
plot_sweep_training(
    file_taxi_conf,
    window_size=500,
    start_episode=0,
    plot_specs=taxi_training_plot_specs(),
)

NameError: name 'plot_sweep_training' is not defined

TypeError: 'PosixPath' object is not iterable

In [ ]:
plot_sweep_evaluation(results, window_size=300, plot_specs=taxi_evaluation_plot_specs())

In [ ]:
# download data for plots
# Taxi - positivity bias
file_taxi_pos = Path("outputs/sweeps/taxi_final_results/taxi_pos_all_ratios.pkl")
with file_taxi_pos.open("rb") as f:
    pos_results = pickle.load(f)
print(f"Loaded {len(pos_results)} runs from {file_taxi_pos}")

## Cliff Walking

In [ ]:
# CliffWalking - positivity bias
file_cliffwalking_pos = Path("outputs/sweeps/cliffwalking_pos_all_ratios.pkl")
with file_cliffwalking_pos.open("rb") as f:
    pos_results = pickle.load(f)
print(f"Loaded {len(pos_results)} runs from {file_cliffwalking_pos}")

# CliffWalking - confirmation bias
file_cliffwalking_conf = Path("outputs/sweeps/cliffwalking_conf_all_pairs.pkl")
with file_cliffwalking_conf.open("rb") as f:
    conf_results = pickle.load(f)
print(f"Loaded {len(conf_results)} runs from {file_cliffwalking_conf}")

## Frozen Lake

In [ ]:
# FrozenLake - positivity bias
file_frozenlake_pos = Path("outputs/sweeps/frozenlake_pos_all_ratios.pkl")
with file_frozenlake_pos.open("rb") as f:
    pos_results = pickle.load(f)
print(f"Loaded {len(pos_results)} runs from {file_frozenlake_pos}")

# FrozenLake - confirmation bias
file_frozenlake_conf = Path("outputs/sweeps/frozenlake_conf_all_pairs.pkl")
with file_frozenlake_conf.open("rb") as f:
    conf_results = pickle.load(f)
print(f"Loaded {len(conf_results)} runs from {file_frozenlake_conf}")